In [1]:
# 1. 의존성 설치
!pip install -q transformers accelerate sacrebleu sentencepiece pandas tabulate huggingface_hub tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 5.4 MB/s eta 0:00:00


In [2]:
import torch
import gc
import time
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer
from sacrebleu.metrics import CHRF, BLEU
from tqdm import tqdm

# GPU 확인
if torch.cuda.is_available():
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
    print(f"📊 VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
else:
    print("⚠️ GPU 없음")

✅ GPU: Tesla T4
📊 VRAM: 14.7 GB


In [ ]:
# -*- coding: utf-8 -*-
"""LFM2 v4/v5/v6 벤치마크 + 문장별 분석 (Colab 자동 다운로드)
노트북 셀에 붙여넣기해서 바로 실행 가능
"""

#==============================================================================
# 📌 1. 의존성 설치 (Colab에서 자동 실행)
#==============================================================================
import sys, subprocess, gc, time, os
if "google.colab" in sys.modules:
    subprocess.run([sys.executable, "-m", "pip", "install", "-q",
                    "transformers", "accelerate", "peft", "sacrebleu", "pandas", "tqdm"])

import torch
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer
from sacrebleu.metrics import CHRF, BLEU
from tqdm import tqdm

#==============================================================================
# � 2. 테스트할 모델 설정
#==============================================================================
MODELS_TO_TEST = [
    # {
    #     "name": "LFM2-v6.3-DPO-Full",
    #     "id": "gyung/lfm2-1.2b-koen-mt-v6.2-dpo",
    #     "type": "transformers"
    # },
    {
          "name": "LFM2-v6.3-DPO-lora",
          "id": "gyung/lfm2-1.2b-koen-mt-v6.1-curriculum",
          "type": "peft_adapter",
          "adapter_id": "gyung/lfm2-1.2b-koen-mt-v6.3-dpo-lora"
    },
    # {
    #     "name": "LFM2-v6.1-curriculum",
    #     "id": "gyung/lfm2-1.2b-koen-mt-v6.1-curriculum",
    #     "type": "transformers"
    # },
    # {
    #     "name": "LFM2-v6-200k",
    #     "id": "gyung/lfm2-1.2b-koen-mt-v6-sft-200k",
    #     "type": "transformers"
    # },
]

#==============================================================================
# 📌 3. 데이터셋 (여기에 데이터 붙여넣기!)
#==============================================================================
# ⬇️⬇️⬇️ 영어 원문 리스트 (ENG_DATA) 여기에 붙여넣기 ⬇️⬇️⬇️
# --- DATASET (Flores-200 DevTest : 1012 samples) ---

ENG_DATA = []
KOR_REF_DATA = []

#==============================================================================
# 📌 4. 핵심 함수들 (수정 불필요)
#==============================================================================
def generate(model, tokenizer, text):
    msgs = [{"role": "system", "content": "Translate the following text to Korean."},
            {"role": "user", "content": text}]
    ids = tokenizer.apply_chat_template(msgs, return_tensors="pt", add_generation_prompt=True).to(model.device)
    with torch.no_grad():
        out = model.generate(ids, max_new_tokens=256, do_sample=True, temperature=0.3,
                             min_p=0.15, repetition_penalty=1.05, pad_token_id=tokenizer.eos_token_id)
    return tokenizer.decode(out[0][ids.shape[1]:], skip_special_tokens=True).strip()

def sentence_scores(trans, refs):
    chrf = CHRF()
    return [round(chrf.sentence_score(t, [r]).score, 2) for t, r in zip(trans, refs)]

def eval_model(info, srcs, refs):
    print(f"\n🔄 Loading {info['name']}...")
    tok = AutoTokenizer.from_pretrained(info["id"])
    if info["type"] == "peft_adapter":
        from peft import PeftModel
        base = AutoModelForCausalLM.from_pretrained(info["id"], device_map="auto", torch_dtype=torch.float16)
        model = PeftModel.from_pretrained(base, info["adapter_id"]).merge_and_unload()
    else:
        model = AutoModelForCausalLM.from_pretrained(info["id"], device_map="auto", torch_dtype=torch.float16)

    trans = [generate(model, tok, s) for s in tqdm(srcs, desc=info["name"])]
    chrf = CHRF().corpus_score(trans, [refs]).score
    bleu = BLEU().corpus_score(trans, [refs]).score
    scores = sentence_scores(trans, refs)

    print(f"✅ {info['name']}: CHrF++={chrf:.2f}, BLEU={bleu:.2f}")
    del model, tok; torch.cuda.empty_cache(); gc.collect()
    return {"name": info["name"], "chrf": chrf, "bleu": bleu, "trans": trans, "scores": scores}

#==============================================================================
# 📌 5. 실행 및 자동 다운로드
#==============================================================================
print(f"📊 Benchmark: {len(ENG_DATA)} samples, {len(MODELS_TO_TEST)} models")
results, data = [], {"Source": ENG_DATA, "Reference": KOR_REF_DATA}

for m in MODELS_TO_TEST:
    r = eval_model(m, ENG_DATA, KOR_REF_DATA)
    results.append(r)
    data[f"Trans_{r['name']}"] = r["trans"]
    data[f"Score_{r['name']}"] = r["scores"]

# Summary
print("\n" + "="*50 + "\n📈 LEADERBOARD\n" + "="*50)
df_sum = pd.DataFrame([{"Model": r["name"], "CHrF++": r["chrf"], "BLEU": r["bleu"]} for r in results])
print(df_sum.sort_values("CHrF++", ascending=False).to_markdown(index=False))

# Save & Auto-download
fname = f"benchmark_{time.strftime('%Y%m%d_%H%M%S')}.csv"
pd.DataFrame(data).to_csv(fname, index=False, encoding='utf-8-sig')
print(f"\n� Saved: {fname}")

if "google.colab" in sys.modules:
    from google.colab import files
    files.download(fname)
    print("📥 자동 다운로드 시작!")


📊 Benchmark: 1012 samples, 1 models

🔄 Loading LFM2-v6.3-DPO-lora...


adapter_model.safetensors:   0%|          | 0.00/22.2M [00:00<?, ?B/s]

LFM2-v6.3-DPO-lora: 100%|██████████| 1012/1012 [16:38<00:00,  1.01it/s]


✅ LFM2-v6.3-DPO-lora: CHrF++=31.46, BLEU=10.90

📈 LEADERBOARD
| Model              |   CHrF++ |    BLEU |
|:-------------------|---------:|--------:|
| LFM2-v6.3-DPO-lora |  31.4637 | 10.8952 |

� Saved: benchmark_20251223_125719.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

📥 자동 다운로드 시작!
